# Inferência

É aplicação do modelo nos patches do conjunto de teste, e avaliação das métricas.

Imports e preparação do device CUDA

In [ ]:
import torch
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))

import src.data.preprocess_data as data
import src.training.train_model as train
import src.data.view as view
import src.models.unets as unets
import src.models.hrnets as hrnets
import src.training.post_processing as post

from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt



def print_gpu_memory(prefix=""):
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"{prefix} Memory Allocated: {allocated:.2f} MB")
        print(f"{prefix} Memory Reserved: {reserved:.2f} MB")
    else:
        print("CUDA is not available.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() 

print_gpu_memory()

### Predição nos conjuntos de teste

Para cada um dos conjuntos de tiles, há a divisão entre treino, validaçao e teste, a inferência utiliza o conjunto de teste de cada.

In [ ]:
tiles_1 = {
              'Belo Horizonte': '032027',
              }

tiles_4 = {
              'Manaus': '016009',
              'Porto Alegre': '025037',
              'Belo Horizonte': '032027',
              'Salvador': '038019',      
              }
tiles_8 = {
              'Boa Vista': '015002',  
              'Campo Grande': '021027',
              'Macapá': '025005',
              'Curitiba': '027032',
              'Brasília': '028022',                      
              'Rio de Janeiro': '033029',
              'Teresina': '034011',
              'Petrolina': '036016',
              }

tiles = {}
tiles['1 tile'] = list(tiles_1.values())
tiles['4 tiles'] = list(tiles_4.values())
tiles['8 tiles'] = list(tiles_8.values())
tiles['12 tiles'] = list(set(list(tiles_4.values())+list(tiles_8.values())))


num_subtiles = 6
classes_mode = '4types'
training_batch_size = 16
model_types = 'unets'
weighted = True

if classes_mode == 'type':
    num_classes = 5
elif classes_mode == 'density':
    num_classes = 4
elif classes_mode == '4types':
    num_classes = 4
elif classes_mode == 'binary':
    num_classes = 2
elif classes_mode == 'all':
    num_classes = 9


working_dir = os.path.abspath('..')
models_paths = os.listdir(os.path.join(working_dir, 'models'))
models_paths = [f for f in models_paths if f.endswith('4tt.pth')]
models_paths = [f for f in models_paths if f.startswith('UNetSmall')]
models_paths.sort()


channels_dict = {}
channels_dict[12] = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A']
#channels_dict[10] = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12', 'B8A']
channels_dict[8] = ['B02', 'B03', 'B04', 'B05', 'B06', 'B08', 'B11', 'B12']
#channels_dict[6] = ['B02', 'B03', 'B04', 'B06', 'B08', 'B11']
channels_dict[4] = ['B02', 'B03', 'B04','B08']


## Modelos escolhidos:

UNetSmall, emm configuração DS-CEW, para 8 e 4 canais, tanto modelo original como finetune.

In [ ]:
working_dir = os.path.abspath('..')
models_paths = os.listdir(os.path.join(working_dir, 'models'))
models_paths += os.listdir(os.path.join(working_dir, 'models', 'finetuned'))
#models_paths = [f for f in models_paths if (f.endswith('8ft.pth') or f.endswith('4tt.pth'))]
models_paths = [f for f in models_paths if f.startswith('UNetSmall')]
models_paths = [f for f in models_paths if 'DS-CEW' in f]
models_paths = [f for f in models_paths if ('-8ch-' in f or '-4ch-' in f)]

models_paths.sort()
models_paths


In [ ]:
import pandas as pd
metrics = pd.DataFrame(columns = ['tile','model', 'metric', 'macro avg', 'weighted avg', 'Class 0', 'Class 1', 'Class 2', 'Class 3'])
metrics.set_index(["tile", "model", "metric"], inplace=True)
metrics

### Predição nos conjuntos de teste

Para cada um dos conjuntos de tiles, há a divisão entre treino, validaçao e teste, a inferência utiliza o conjunto de teste de cada.

In [ ]:

for tile_set in tiles:
    
    train_files, val_files, test_files = data.train_val_test_stratify(tiles[tile_set], 
                                                                    num_subtiles,
                                                                    train_size = 0.6, 
                                                                    val_size = 0.2, 
                                                                    stratify_by = classes_mode,
                                                                    subfolder='q_12ch')

    class_labels = list(range(num_classes))

    for model_name in models_paths:# if 'UNet-256-type-DS-CEW' in mp]:#model_paths:
    #for model_name in model_paths:#[mp for mp in models_paths if 'UNet-256-type-DS-CEW' in mp]:#model_paths:
        print('Model name:', model_name)
        #UNet-256-4types-DS-CE-12ch-4tt.pth
        import re
        match = re.search(r"(\d+)ch", model_name)
        if match:
            in_channels = int(match.group(1))  # Saída: 12
            print('Input channels: ', in_channels)
        print('Num classes:', num_classes)
        if model_name.startswith('UNetSmall-'):
            model = unets.UNetSmall(in_channels=in_channels, out_channels=num_classes).to(device) 
        elif model_name.startswith('UNet-'):
            model = unets.UNet(in_channels=in_channels, out_channels=num_classes).to(device) 
        elif model_name.startswith('UNetResNet34-'):
            model = unets.UNetResNet34(in_channels=in_channels, out_channels=num_classes).to(device) 
        elif model_name.startswith('UNetEfficientNetB0-'):
            model = unets.UNetEfficientNetB0(in_channels=in_channels, out_channels=num_classes).to(device) 
        elif model_name.startswith('UNetConvNext-'):
            model = unets.UNetConvNext(in_channels=in_channels, out_channels=num_classes).to(device) 
        elif model_name.startswith('HRNetW18'):
            model = hrnets.HRNetSegmentation(in_channels= in_channels, num_classes=num_classes, backbone="hrnet_w18_small", pretrained=True,).to(device)
        elif model_name.startswith('HRNetW32'):
            model = hrnets.HRNetSegmentation(in_channels= in_channels, num_classes=num_classes, backbone="hrnet_w32", pretrained=True,).to(device)
        elif model_name.startswith('HRNetW48'):
            model = hrnets.HRNetSegmentation(in_channels= in_channels, num_classes=num_classes, backbone="hrnet_w48", pretrained=True,).to(device)
        else:
            print('Nao existe esse modelo')
            continue
        if 'finetuned' in model_name:
            checkpoint = torch.load(os.path.join(working_dir, 'models', 'finetuned', model_name), weights_only=False)
        else:
            checkpoint = torch.load(os.path.join(working_dir, 'models', model_name), weights_only=False)


        
        indices = [i for i, value in enumerate(channels_dict[in_channels]) if value in channels_dict[12]]

        yaml_filename = data.yaml_filename(num_subtiles, tiles[tile_set], classes_mode)
        print(yaml_filename)
        print('----------------')
        patch_size = int(model_name.split('-')[1])
        
        test_dataset = data.SubtileDataset(yaml_filename, 
                                        set = 'test_files',
                                        patch_size=patch_size, 
                                        stride=patch_size, 
                                        dynamic_sampling = False,
                                        data_augmentation = False, # testando 
                                        channels_subset = indices
                                        )
        BS = 16
        dataloader = DataLoader(test_dataset, batch_size=BS, shuffle=False)

        print(model_name)
        recalls, report = train.test_model(model, 
                        checkpoint_path=model_name,
                        dataloader = dataloader, 
                        device = device, 
                        num_classes = num_classes,
                        subfolder= 'finetuned' if 'finetuned' in model_name else '',
                        set_name = f'-test-{tile_set}'
                        ) 

        metrics.loc[(tile_set, model_name, "f1-score"), 'macro avg'] = report['f1-score']['macro avg']
        metrics.loc[(tile_set, model_name, "f1-score"), 'weighted avg'] = report['f1-score']['weighted avg']
        for i, cl in enumerate(class_labels):
            metrics.loc[(tile_set, model_name, "f1-score"), f'Class {i}'] = report['f1-score'][f'Class {i}']

        metrics.loc[(tile_set, model_name, "precision"), 'macro avg'] = report['precision']['macro avg']
        metrics.loc[(tile_set, model_name, "precision"), 'weighted avg'] = report['precision']['weighted avg']
        for i, cl in enumerate(class_labels):
            metrics.loc[(tile_set, model_name, "precision"), f'Class {i}'] = report['precision'][f'Class {i}']

        metrics.loc[(tile_set, model_name, "recall"), 'macro avg'] = report['recall']['macro avg']
        metrics.loc[(tile_set, model_name, "recall"), 'weighted avg'] = report['recall']['weighted avg']
        for i, cl in enumerate(class_labels):
            metrics.loc[(tile_set, model_name, "recall"), f'Class {i}'] = report['recall'][f'Class {i}']



# Visualização dos resultados

metrics contém a tabela de resultados

In [ ]:
metrics

In [ ]:
metrics_percent = metrics.applymap(lambda x: f"{x * 100:.1f}")
metrics_percent


In [ ]:
print(metrics_percent.to_latex())

In [ ]:
for tile_set in tiles:
    print(tiles[tile_set])